In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import re

In [ ]:
# 001005 - 맨투맨, 001002 - 셔츠, 001004 - 후드티, 001010 - 긴소매, 001001 - 반소매

product_url = "https://display.musinsa.com/display/api/v2/categories/ITEM/goods"

review_url = "https://goods.musinsa.com/api/goods/v2/review-list-html"
header = header

link = link

# 카테고리별 상품 크롤링

In [ ]:
def cate_crawling(cate, page):
    product_params = {
        "siteKindId": "musinsa",
        "sex": "M",
        "sortCode": "1y",
        "categoryCode": cate,
        "size": "100",
        "page": page
    }

    # 정보 요청
    response = requests.get(product_url, params = product_params)

    # 성공 / 실패 여부 print
    if response.status_code == 200:
        print(f"{cate} 상품정보 성공")
    else:
        print(f"{cate} 상품정보 실패")

    # 가져온 데이터를 json 형식으로 변환
    data = response.json()

    product_df = pd.DataFrame(data["data"]["goodsList"])
    new_df = product_df[["goodsNo", "brandName", "brandNameEng", "goodsName", "imageUrl"
                   ,"relatedGoodsReviewScore"]]
    return new_df

In [ ]:
cate_df = cate_crawling(cate = "001005", page = "3")

001005 상품정보 성공


In [ ]:
# 001005 - 맨투맨, 001002 - 셔츠, 001004 - 후드티, 001010 - 긴소매, 001001 - 반소매
cate_list = ["001005", "001002", "001004", "001010"]

for cate in cate_list:
    for num in range(1, 4):
        cate_df = cate_crawling(cate = f"{cate}", page = f"{num}")
        cate_df.to_csv(f"{link}product_cate/{cate}_{num}page.csv", index = False)
        print(f"{num}페이지 수집")

001005 상품정보 성공
1페이지 수집
001005 상품정보 성공
2페이지 수집
001005 상품정보 성공
3페이지 수집
001002 상품정보 성공
1페이지 수집
001002 상품정보 성공
2페이지 수집
001002 상품정보 성공
3페이지 수집
001004 상품정보 성공
1페이지 수집
001004 상품정보 성공
2페이지 수집
001004 상품정보 성공
3페이지 수집
001010 상품정보 성공
1페이지 수집
001010 상품정보 성공
2페이지 수집
001010 상품정보 성공
3페이지 수집


# 리뷰 데이터 크롤링

In [ ]:
new_df = pd.read_csv(f"{link}product_cate/001002_1page.csv")

In [ ]:
def fetch_reviews(goods_no, page_num, header, review_url):
    review_params = {
        "goodsNo": f"{goods_no}",
        "sex": "",
        "myFilter": "false",
        "type": "goods",
        "bodyFilterAvailable": "Y",
        "similarNo": "0",
        "selectedSimilarNo": f"{goods_no}",
        "selectedSimilarName": "",
        "minHeight": "0",
        "maxHeight": "0",
        "minWeight": "0",
        "maxWeight": "0",
        "minHeightList": "",
        "maxHeightList": "",
        "minWeightList": "",
        "maxWeightList": "",
        "skinType": "",
        "skinTone": "",
        "skinWorry": "",
        "reviewSex": "W",
        "optionCnt": "1",
        "option1": "",
        "option2": "",
        "option1List": "",
        "option2List": "",
        "keyword": "",
        "page": f"{page_num}",
        "sort": "up_cnt_desc"
    }

    response = requests.get(review_url, params = review_params, headers = header)
    if response.status_code == 200:
        print(f"{goods_no}상품 - {page_num}페이지 실행")
        return BeautifulSoup(response.text, "lxml")
    else:
        print(f"{goods_no} 상품 리뷰 불러오기 실패")
        return None

def parse_reviews(soup, goods_no):
    review_profiles = soup.select("div.review-profile__text")
    review_informations = soup.select("div.review-profile__information")
    review_contents = soup.select("div.review-contents__text")
    review_type_class = soup.select("div.review-evaluation--type2 ul")
    review_items = soup.select("div.review-goods-information__item")
    review_sizes = [item.select_one("span").text.strip() for item in review_items]

    reviews = []

    for review_num in range(len(review_profiles)):
        try:
            review_info_dict = {
                "goodsNo": goods_no,
                "id_date": review_profiles[review_num].text,
                "body_size": review_informations[review_num].text,
                "review": review_contents[review_num].text,
                "buy_size": review_sizes[review_num],
                "type_class" : review_type_class[review_num].text.strip().split("\n")[0]
            }
        except:
            review_info_dict = {
                "goodsNo": goods_no,
                "id_date": review_profiles[review_num].text,
                "body_size": review_informations[review_num].text,
                "review": review_contents[review_num].text,
                "buy_size": review_sizes[review_num],
                "type_class" : "x"
            }
        reviews.append(review_info_dict)
    return reviews

def review_crawling(df, num):
    review_info = []

    try:
        for i in df:
            for j in range(1, num):
                soup = fetch_reviews(i, j, header, review_url)
                if soup:
                    review_info.extend(parse_reviews(soup, i))

    except Exception as e:
        print(e)
        temp_df = pd.DataFrame(review_info)
        temp_df.to_csv("temp.csv", index = False)

    return review_info

In [ ]:
review_df = pd.DataFrame(review_crawling(new_df["goodsNo"], 31))

2272830상품 - 1페이지 실행
2272830상품 - 2페이지 실행
2272830상품 - 3페이지 실행
2272830상품 - 4페이지 실행
2272830상품 - 5페이지 실행
2272830상품 - 6페이지 실행
2272830상품 - 7페이지 실행
2272830상품 - 8페이지 실행
2272830상품 - 9페이지 실행
2272830상품 - 10페이지 실행
2272830상품 - 11페이지 실행
2272830상품 - 12페이지 실행
2272830상품 - 13페이지 실행
2272830상품 - 14페이지 실행
2272830상품 - 15페이지 실행
2272830상품 - 16페이지 실행
2272830상품 - 17페이지 실행
2272830상품 - 18페이지 실행
2272830상품 - 19페이지 실행
2272830상품 - 20페이지 실행
2272830상품 - 21페이지 실행
2272830상품 - 22페이지 실행
2272830상품 - 23페이지 실행
2272830상품 - 24페이지 실행
2272830상품 - 25페이지 실행
2272830상품 - 26페이지 실행
2272830상품 - 27페이지 실행
2272830상품 - 28페이지 실행
2272830상품 - 29페이지 실행
2272830상품 - 30페이지 실행
994588상품 - 1페이지 실행
994588상품 - 2페이지 실행
994588상품 - 3페이지 실행
994588상품 - 4페이지 실행
994588상품 - 5페이지 실행
994588상품 - 6페이지 실행
994588상품 - 7페이지 실행
994588상품 - 8페이지 실행
994588상품 - 9페이지 실행
994588상품 - 10페이지 실행
994588상품 - 11페이지 실행
994588상품 - 12페이지 실행
994588상품 - 13페이지 실행
994588상품 - 14페이지 실행
994588상품 - 15페이지 실행
994588상품 - 16페이지 실행
994588상품 - 17페이지 실행
994588상품 - 18페이지 실행
994588상품 - 19페이지 실행
994588상품

In [ ]:
review_df.to_csv(f"{link}product_review/001002_review_1page.csv", index = False)

In [ ]:
review_df.shape

(29517, 6)

# 사이즈 크롤링

In [ ]:
import requests
import time

def size_crawling(df):
    size_data = []
    exceptions = []

    for num in df:
        try:
            size_url = f"https://goods-detail.musinsa.com/goods/{num}/actual-size"
            size_frame = requests.get(size_url, headers=header)
            size_frame.raise_for_status()  # HTTPError를 발생시키기 위해 추가
            size_infos = size_frame.json()["data"]["sizes"]

            for size in size_infos:
                goodsNo = num
                Size = size["name"]
                temp_dict = {"goodsNo": goodsNo, "Size": Size}
                for item in size["items"]:
                    temp_dict[item["name"]] = item["value"]
                size_data.append(temp_dict)

        except Exception as e:
            exceptions.append((e, num))  # 예외를 리스트에 기록
            pass  # 예외가 발생했을 때 아무 작업도 수행하지 않음

        # 각 요청 사이에 1초의 지연을 추가
        time.sleep(1)

    # 예외 목록 출력
    for e, num in exceptions:
        print(f"Exception for goodsNo {num}: {e}")

    return size_data

In [ ]:
new_df = pd.read_csv(f"{link}product_cate/001005_1page.csv")

In [ ]:
size_df = pd.DataFrame(size_crawling(new_df["goodsNo"]))

In [ ]:
size_df.to_csv(f"{link}product_size/001005_size_1page.csv", index = False)

In [ ]:
size_df.shape

(389, 6)